In [16]:
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
S = PowerSeriesRing(QQ,'x', default_prec=300)
L=LaurentSeriesRing(QQ,'x', default_prec=300)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=300)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.

def uniteList(lst):
    ln=len(lst)
    if ln<2:return lst
    answer=[]
    for j in [0..ln-1]:
        lj=lst[j]
        if (lj in answer)==False:
            answer=answer+[lj]
    return answer

def delete_element(elt,ls):
    new=[]
    for j in ls:
        if j!=elt:new=new+[j]
    return new

def sortAndSmash(lst): #sorts and deletes repeats
    cropped=lst
    new=[]
    while len(cropped)>0:
        m=min(cropped)
        new=new+[m]
        cropped=delete_element(m,cropped)
    return(new)

def difference_table(lst):
    diffs=[]
    for k in [0..len(lst)-2]:
        diff=lst[k+1]-lst[k]
        diffs=diffs+[diff]
    return diffs

def evaluate(expression,incomingvalue):
    expr=str(expression)
    output=sage_eval(expr,locals={'x':incomingvalue})
    return output

P=Primes()

def prime(n): return P.unrank(n-1)

def unite(A,B):
    return set(A).union(B)

def factor_integer(n): #this is like "factor(integer)", but
    #the bases and exponents can be easily extracted.
    F=factor(n)
    return list(F)

def orderInteger(p,n):
    if n!=0:
        lst=n.digits(p)
        count=0
        k=0
        while lst[k]==0:
            count=count+1
            k=k+1
        return count
    if n==0:return(Infinity)

def order(p,fr):
    nn=numerator(QQ(fr))
    dn=denominator(QQ(fr))
    return orderInteger(p,nn)-orderInteger(p,dn)

def digitsum(p,n):return add(n.digits(p))

def digitsum(p,n):
    return sum(n.digits(p))

def rmnjntau(n):
    answer=0
    for k in [1..n-1]:
        answer=answer+(6*k-4*(n-k))*sigma(k,3)*sigma(n-k,5)
    answer=(5*sigma(n,3)+7*sigma(n,5))*n/12-35*answer
    return answer

def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])

def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer

def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))

def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))

def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)

def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))

def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)

def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)

def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # 
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.

def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)

def Jcoefficient(n,m):
    f = J(n+2,m)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]

def xjNew(n,m):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(m**3*2**6)**i
        return rt
    return xjNewStep(n-1,m)

def j(n,m):return L(xjNew(n,m)/x)

def xjLeoNew(n,m,multiplier):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(multiplier)**i
        return rt
    return xjNewStep(n-1,m)

def jLeo(n,m,multiplier):return L(xjLeoNew(n,m,multiplier)/x)

def JpowerConstantTerm(m,power):
    trun=J(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def jpowerConstantTerm(m,power):
    trun=j(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def polynomialCoefficient(n,poly):
    return poly.list()[n]

def leadingCoefficient(polynomial):
    dg=polynomial.degree()
    
    return polynomialCoefficient(dg,polynomial)

def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent

def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)

def firstfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.first())

def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())

def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)

def element(k,listo):
    if k>len(listo):return (print("bad k"))
    return listo[k-1]

def mobiusInverseOfFunction(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        f=f+moebius(dvk)*g(n/dvk)
    return f

def mobiusInverseOfList(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        gndvk=element(n/dvk,g)
        f=f+moebius(dvk)*gndvk
    return f

def drop_first_element(listo):
    lenth=len(listo)
    answer=[]
    for k in [1..lenth-1]:
        answer=answer+[listo[k]]
    return answer

def factorSeriesFromLists(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
    return nswr

def factorSeriesFromListsWithTracking(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
            print((n,k))
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
        print("answer length: ",len(nswr))
    return nswr

def EulerMultiplication(listo):
    # Lemma 2.11 from my 1998 Experimental Mathematics paper.
    # Here the set A in the lemma is {1, 2, ...} = Z^+.
    p_A=[1] # so p_A[0]=1 as on top of p. 266, column 2.
    def ex(n):return listo[n-1] # ex is the exponent on the factor 1-x^n: -f(n)/n.
    def f(n):return -n*ex(n)
    answer=[1]
    def f_A(k):
        summ=0
        dk=divisors(k)
        for d in dk: #Here is where A = Z^+ is used to drop a constraint on k.
            summ=summ+f(d)
        return summ
    for n in [1..len(listo)-1]:
        sm=0
        for k in [1..n]:
            sm=sm+f_A(k)*p_A[n-k]
        p_A=p_A+[sm/n]
    return p_A

def listToPoly(listo):
    lenth=len(listo)
    answer=0
    for k in [1..lenth]:
        cf=listo[k-1]
        answer=answer+cf*x^(k-1)
    return answer
    


def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})

print("okay1")

okay1


In [1]:
sigma?

for n in [1..3]:


In [8]:
for n in [1..5]:
    print((n,sigma(n,k=11)))

(1, 1)
(2, 2049)
(3, 177148)
(4, 4196353)
(5, 48828126)


In [9]:
L=LaurentSeriesRing(QQ,'x', default_prec=1050)

def p2RamanujanCongruenceSeries(n): # Ramanujan Revisted, p. 290
    ans=0
    for a in [1..n+3]:
        if mod(a,2)==0:term=0
        if mod(a,2)==1:term=sigma(a,k=11)*x^a
        ans=ans+term
    return ans

def recipP2(n):
    return L(1/p2RamanujanCongruenceSeries(n)).O(n+1)

def p2RecipPowerConstantTerm(k):
    ans=recipP2(k)^k
    polyversion=expand(ans*x^k)
    constant=polynomialCoefficient(k,polyversion)
    return constant
    
lst2=[]
zs2=[]
nzs2=[]
for n in [1..1000]:
    c=p2RecipPowerConstantTerm(n)
    if c==0:
        zs2=zs2+[mod(n,2)]
    if c!=0:
        nzs2=nzs2+[mod(n,2)]
        o2=order(2,c)
        sd2=digitsum(2,n)
        test2=3*sd2-o2
        lst2=lst+[test2]
        print((n,test2))
print("---------------------------------------------------------------------------------------------------")
print("lst2:");print(lst2);print();print("zs2:");print(zs2);print();print("nzs2:");print(nzs2)

(2, 0)
(4, 0)
(6, 0)
(8, 0)
(10, 0)
(12, 0)
(14, 0)
(16, 0)
(18, 0)
(20, 0)
(22, 0)
(24, 0)
(26, 0)
(28, 0)
(30, 0)
(32, 0)
(34, 0)
(36, 0)
(38, 0)
(40, 0)
(42, 0)
(44, 0)
(46, 0)
(48, 0)
(50, 0)
(52, 0)
(54, 0)
(56, 0)
(58, 0)
(60, 0)
(62, 0)
(64, 0)
(66, 0)
(68, 0)
(70, 0)
(72, 0)
(74, 0)
(76, 0)
(78, 0)
(80, 0)
(82, 0)
(84, 0)
(86, 0)
(88, 0)
(90, 0)
(92, 0)
(94, 0)
(96, 0)
(98, 0)
(100, 0)
(102, 0)
(104, 0)
(106, 0)
(108, 0)
(110, 0)
(112, 0)
(114, 0)
(116, 0)
(118, 0)
(120, 0)
(122, 0)
(124, 0)
(126, 0)
(128, 0)
(130, 0)
(132, 0)
(134, 0)
(136, 0)
(138, 0)
(140, 0)
(142, 0)
(144, 0)
(146, 0)
(148, 0)
(150, 0)
(152, 0)
(154, 0)
(156, 0)
(158, 0)
(160, 0)
(162, 0)
(164, 0)
(166, 0)
(168, 0)
(170, 0)
(172, 0)
(174, 0)
(176, 0)
(178, 0)
(180, 0)
(182, 0)
(184, 0)
(186, 0)
(188, 0)
(190, 0)
(192, 0)
(194, 0)
(196, 0)
(198, 0)
(200, 0)
(202, 0)
(204, 0)
(206, 0)
(208, 0)
(210, 0)
(212, 0)
(214, 0)
(216, 0)
(218, 0)
(220, 0)
(222, 0)
(224, 0)
(226, 0)
(228, 0)
(230, 0)
(232, 0)
(234, 0)


In [19]:
L=LaurentSeriesRing(QQ,'x', default_prec=1050)
import random

def specialModulus(a):
    ds=digitsum(2,a)
    return 2^20*8^ds

def p2RamanujanCongruenceSeries(n): # Ramanujan Revisted, p. 290
    ans=0
    for a in [1..n+3]:
        sm=specialModulus(a)
        r=ZZ(random.randint(-10, 10))
        if mod(a,2)==0:term=0
        if mod(a,2)==1:term=sigma(a,k=11)+r*sm
        ans=ans+term*x^a
    return ans

def recipP2(n):
    return L(1/p2RamanujanCongruenceSeries(n)).O(n+1)

def p2RecipPowerConstantTerm(k):
    ans=recipP2(k)^k
    polyversion=expand(ans*x^k)
    constant=polynomialCoefficient(k,polyversion)
    return constant
    
lst2=[]
zs2=[]
nzs2=[]
for trial in [1..10]:
    for n in [1..1000]:
        c=p2RecipPowerConstantTerm(n)
        if c==0:
            zs2=zs2+[mod(n,2)]
        if c!=0:
            nzs2=nzs2+[mod(n,2)]
            o2=order(2,c)
            sd2=digitsum(2,n)
            test2=3*sd2-o2
            lst2=lst2+[test2]
            print((trial,n,test2))
    print("----------------------------------------------------------------------------------------------")
    print("trial:",trial);print()
    print("lst2:");print(lst2);print();print("zs2:");print(zs2);print();print("nzs2:");print(nzs2)

(1, 2, 0)
(1, 4, 0)
(1, 6, 0)
(1, 8, 0)
(1, 10, 0)
(1, 12, 0)
(1, 14, 0)
(1, 16, 0)
(1, 18, 0)
(1, 20, 0)
(1, 22, 0)
(1, 24, 0)
(1, 26, 0)
(1, 28, 0)
(1, 30, 0)
(1, 32, 0)
(1, 34, 0)
(1, 36, 0)
(1, 38, 0)
(1, 40, 0)
(1, 42, 0)
(1, 44, 0)
(1, 46, 0)
(1, 48, 0)
(1, 50, 0)
(1, 52, 0)
(1, 54, 0)
(1, 56, 0)
(1, 58, 0)
(1, 60, 0)
(1, 62, 0)
(1, 64, 0)
(1, 66, 0)
(1, 68, 0)
(1, 70, 0)
(1, 72, 0)
(1, 74, 0)
(1, 76, 0)
(1, 78, 0)
(1, 80, 0)
(1, 82, 0)
(1, 84, 0)
(1, 86, 0)
(1, 88, 0)
(1, 90, 0)
(1, 92, 0)
(1, 94, 0)
(1, 96, 0)
(1, 98, 0)
(1, 100, 0)
(1, 102, 0)
(1, 104, 0)
(1, 106, 0)
(1, 108, 0)
(1, 110, 0)
(1, 112, 0)
(1, 114, 0)
(1, 116, 0)
(1, 118, 0)
(1, 120, 0)
(1, 122, 0)
(1, 124, 0)
(1, 126, 0)
(1, 128, 0)
(1, 130, 0)
(1, 132, 0)
(1, 134, 0)
(1, 136, 0)
(1, 138, 0)
(1, 140, 0)
(1, 142, 0)
(1, 144, 0)
(1, 146, 0)
(1, 148, 0)
(1, 150, 0)
(1, 152, 0)
(1, 154, 0)
(1, 156, 0)
(1, 158, 0)
(1, 160, 0)
(1, 162, 0)
(1, 164, 0)
(1, 166, 0)
(1, 168, 0)
(1, 170, 0)
(1, 172, 0)
(1, 174, 0)
(1, 176, 

(2, 4, 0)
(2, 6, 0)
(2, 8, 0)
(2, 10, 0)
(2, 12, 0)
(2, 14, 0)
(2, 16, 0)
(2, 18, 0)
(2, 20, 0)
(2, 22, 0)
(2, 24, 0)
(2, 26, 0)
(2, 28, 0)
(2, 30, 0)
(2, 32, 0)
(2, 34, 0)
(2, 36, 0)
(2, 38, 0)
(2, 40, 0)
(2, 42, 0)
(2, 44, 0)
(2, 46, 0)
(2, 48, 0)
(2, 50, 0)
(2, 52, 0)
(2, 54, 0)
(2, 56, 0)
(2, 58, 0)
(2, 60, 0)
(2, 62, 0)
(2, 64, 0)
(2, 66, 0)
(2, 68, 0)
(2, 70, 0)
(2, 72, 0)
(2, 74, 0)
(2, 76, 0)
(2, 78, 0)
(2, 80, 0)
(2, 82, 0)
(2, 84, 0)
(2, 86, 0)
(2, 88, 0)
(2, 90, 0)
(2, 92, 0)
(2, 94, 0)
(2, 96, 0)
(2, 98, 0)
(2, 100, 0)
(2, 102, 0)
(2, 104, 0)
(2, 106, 0)
(2, 108, 0)
(2, 110, 0)
(2, 112, 0)
(2, 114, 0)
(2, 116, 0)
(2, 118, 0)
(2, 120, 0)
(2, 122, 0)
(2, 124, 0)
(2, 126, 0)
(2, 128, 0)
(2, 130, 0)
(2, 132, 0)
(2, 134, 0)
(2, 136, 0)
(2, 138, 0)
(2, 140, 0)
(2, 142, 0)
(2, 144, 0)
(2, 146, 0)
(2, 148, 0)
(2, 150, 0)
(2, 152, 0)
(2, 154, 0)
(2, 156, 0)
(2, 158, 0)
(2, 160, 0)
(2, 162, 0)
(2, 164, 0)
(2, 166, 0)
(2, 168, 0)
(2, 170, 0)
(2, 172, 0)
(2, 174, 0)
(2, 176, 0)
(2, 178

(3, 4, 0)
(3, 6, 0)
(3, 8, 0)
(3, 10, 0)
(3, 12, 0)
(3, 14, 0)
(3, 16, 0)
(3, 18, 0)
(3, 20, 0)
(3, 22, 0)
(3, 24, 0)
(3, 26, 0)
(3, 28, 0)
(3, 30, 0)
(3, 32, 0)
(3, 34, 0)
(3, 36, 0)
(3, 38, 0)
(3, 40, 0)
(3, 42, 0)
(3, 44, 0)
(3, 46, 0)
(3, 48, 0)
(3, 50, 0)
(3, 52, 0)
(3, 54, 0)
(3, 56, 0)
(3, 58, 0)
(3, 60, 0)
(3, 62, 0)
(3, 64, 0)
(3, 66, 0)
(3, 68, 0)
(3, 70, 0)
(3, 72, 0)
(3, 74, 0)
(3, 76, 0)
(3, 78, 0)
(3, 80, 0)
(3, 82, 0)
(3, 84, 0)
(3, 86, 0)
(3, 88, 0)
(3, 90, 0)
(3, 92, 0)
(3, 94, 0)
(3, 96, 0)
(3, 98, 0)
(3, 100, 0)
(3, 102, 0)
(3, 104, 0)
(3, 106, 0)
(3, 108, 0)
(3, 110, 0)
(3, 112, 0)
(3, 114, 0)
(3, 116, 0)
(3, 118, 0)
(3, 120, 0)
(3, 122, 0)
(3, 124, 0)
(3, 126, 0)
(3, 128, 0)
(3, 130, 0)
(3, 132, 0)
(3, 134, 0)
(3, 136, 0)
(3, 138, 0)
(3, 140, 0)
(3, 142, 0)
(3, 144, 0)
(3, 146, 0)
(3, 148, 0)
(3, 150, 0)
(3, 152, 0)
(3, 154, 0)
(3, 156, 0)
(3, 158, 0)
(3, 160, 0)
(3, 162, 0)
(3, 164, 0)
(3, 166, 0)
(3, 168, 0)
(3, 170, 0)
(3, 172, 0)
(3, 174, 0)
(3, 176, 0)
(3, 178

(4, 6, 0)
(4, 8, 0)
(4, 10, 0)
(4, 12, 0)
(4, 14, 0)
(4, 16, 0)
(4, 18, 0)
(4, 20, 0)
(4, 22, 0)
(4, 24, 0)
(4, 26, 0)
(4, 28, 0)
(4, 30, 0)
(4, 32, 0)
(4, 34, 0)
(4, 36, 0)
(4, 38, 0)
(4, 40, 0)
(4, 42, 0)
(4, 44, 0)
(4, 46, 0)
(4, 48, 0)
(4, 50, 0)
(4, 52, 0)
(4, 54, 0)
(4, 56, 0)
(4, 58, 0)
(4, 60, 0)
(4, 62, 0)
(4, 64, 0)
(4, 66, 0)
(4, 68, 0)
(4, 70, 0)
(4, 72, 0)
(4, 74, 0)
(4, 76, 0)
(4, 78, 0)
(4, 80, 0)
(4, 82, 0)
(4, 84, 0)
(4, 86, 0)
(4, 88, 0)
(4, 90, 0)
(4, 92, 0)
(4, 94, 0)
(4, 96, 0)
(4, 98, 0)
(4, 100, 0)
(4, 102, 0)
(4, 104, 0)
(4, 106, 0)
(4, 108, 0)
(4, 110, 0)
(4, 112, 0)
(4, 114, 0)
(4, 116, 0)
(4, 118, 0)
(4, 120, 0)
(4, 122, 0)
(4, 124, 0)
(4, 126, 0)
(4, 128, 0)
(4, 130, 0)
(4, 132, 0)
(4, 134, 0)
(4, 136, 0)
(4, 138, 0)
(4, 140, 0)
(4, 142, 0)
(4, 144, 0)
(4, 146, 0)
(4, 148, 0)
(4, 150, 0)
(4, 152, 0)
(4, 154, 0)
(4, 156, 0)
(4, 158, 0)
(4, 160, 0)
(4, 162, 0)
(4, 164, 0)
(4, 166, 0)
(4, 168, 0)
(4, 170, 0)
(4, 172, 0)
(4, 174, 0)
(4, 176, 0)
(4, 178, 0)
(4, 1

(5, 4, 0)
(5, 6, 0)
(5, 8, 0)
(5, 10, 0)
(5, 12, 0)
(5, 14, 0)
(5, 16, 0)
(5, 18, 0)
(5, 20, 0)
(5, 22, 0)
(5, 24, 0)
(5, 26, 0)
(5, 28, 0)
(5, 30, 0)
(5, 32, 0)
(5, 34, 0)
(5, 36, 0)
(5, 38, 0)
(5, 40, 0)
(5, 42, 0)
(5, 44, 0)
(5, 46, 0)
(5, 48, 0)
(5, 50, 0)
(5, 52, 0)
(5, 54, 0)
(5, 56, 0)
(5, 58, 0)
(5, 60, 0)
(5, 62, 0)
(5, 64, 0)
(5, 66, 0)
(5, 68, 0)
(5, 70, 0)
(5, 72, 0)
(5, 74, 0)
(5, 76, 0)
(5, 78, 0)
(5, 80, 0)
(5, 82, 0)
(5, 84, 0)
(5, 86, 0)
(5, 88, 0)
(5, 90, 0)
(5, 92, 0)
(5, 94, 0)
(5, 96, 0)
(5, 98, 0)
(5, 100, 0)
(5, 102, 0)
(5, 104, 0)
(5, 106, 0)
(5, 108, 0)
(5, 110, 0)
(5, 112, 0)
(5, 114, 0)
(5, 116, 0)
(5, 118, 0)
(5, 120, 0)
(5, 122, 0)
(5, 124, 0)
(5, 126, 0)
(5, 128, 0)
(5, 130, 0)
(5, 132, 0)
(5, 134, 0)
(5, 136, 0)
(5, 138, 0)
(5, 140, 0)
(5, 142, 0)
(5, 144, 0)
(5, 146, 0)
(5, 148, 0)
(5, 150, 0)
(5, 152, 0)
(5, 154, 0)
(5, 156, 0)
(5, 158, 0)
(5, 160, 0)
(5, 162, 0)
(5, 164, 0)
(5, 166, 0)
(5, 168, 0)
(5, 170, 0)
(5, 172, 0)
(5, 174, 0)
(5, 176, 0)
(5, 178

(6, 4, 0)
(6, 6, 0)
(6, 8, 0)
(6, 10, 0)
(6, 12, 0)
(6, 14, 0)
(6, 16, 0)
(6, 18, 0)
(6, 20, 0)
(6, 22, 0)
(6, 24, 0)
(6, 26, 0)
(6, 28, 0)
(6, 30, 0)
(6, 32, 0)
(6, 34, 0)
(6, 36, 0)
(6, 38, 0)
(6, 40, 0)
(6, 42, 0)
(6, 44, 0)
(6, 46, 0)
(6, 48, 0)
(6, 50, 0)
(6, 52, 0)
(6, 54, 0)
(6, 56, 0)
(6, 58, 0)
(6, 60, 0)
(6, 62, 0)
(6, 64, 0)
(6, 66, 0)
(6, 68, 0)
(6, 70, 0)
(6, 72, 0)
(6, 74, 0)
(6, 76, 0)
(6, 78, 0)
(6, 80, 0)
(6, 82, 0)
(6, 84, 0)
(6, 86, 0)
(6, 88, 0)
(6, 90, 0)
(6, 92, 0)
(6, 94, 0)
(6, 96, 0)
(6, 98, 0)
(6, 100, 0)
(6, 102, 0)
(6, 104, 0)
(6, 106, 0)
(6, 108, 0)
(6, 110, 0)
(6, 112, 0)
(6, 114, 0)
(6, 116, 0)
(6, 118, 0)
(6, 120, 0)
(6, 122, 0)
(6, 124, 0)
(6, 126, 0)
(6, 128, 0)
(6, 130, 0)
(6, 132, 0)
(6, 134, 0)
(6, 136, 0)
(6, 138, 0)
(6, 140, 0)
(6, 142, 0)
(6, 144, 0)
(6, 146, 0)
(6, 148, 0)
(6, 150, 0)
(6, 152, 0)
(6, 154, 0)
(6, 156, 0)
(6, 158, 0)
(6, 160, 0)
(6, 162, 0)
(6, 164, 0)
(6, 166, 0)
(6, 168, 0)
(6, 170, 0)
(6, 172, 0)
(6, 174, 0)
(6, 176, 0)
(6, 178

(7, 4, 0)
(7, 6, 0)
(7, 8, 0)
(7, 10, 0)
(7, 12, 0)
(7, 14, 0)
(7, 16, 0)
(7, 18, 0)
(7, 20, 0)
(7, 22, 0)
(7, 24, 0)
(7, 26, 0)
(7, 28, 0)
(7, 30, 0)
(7, 32, 0)
(7, 34, 0)
(7, 36, 0)
(7, 38, 0)
(7, 40, 0)
(7, 42, 0)
(7, 44, 0)
(7, 46, 0)
(7, 48, 0)
(7, 50, 0)
(7, 52, 0)
(7, 54, 0)
(7, 56, 0)
(7, 58, 0)
(7, 60, 0)
(7, 62, 0)
(7, 64, 0)
(7, 66, 0)
(7, 68, 0)
(7, 70, 0)
(7, 72, 0)
(7, 74, 0)
(7, 76, 0)
(7, 78, 0)
(7, 80, 0)
(7, 82, 0)
(7, 84, 0)
(7, 86, 0)
(7, 88, 0)
(7, 90, 0)
(7, 92, 0)
(7, 94, 0)
(7, 96, 0)
(7, 98, 0)
(7, 100, 0)
(7, 102, 0)
(7, 104, 0)
(7, 106, 0)
(7, 108, 0)
(7, 110, 0)
(7, 112, 0)
(7, 114, 0)
(7, 116, 0)
(7, 118, 0)
(7, 120, 0)
(7, 122, 0)
(7, 124, 0)
(7, 126, 0)
(7, 128, 0)
(7, 130, 0)
(7, 132, 0)
(7, 134, 0)
(7, 136, 0)
(7, 138, 0)
(7, 140, 0)
(7, 142, 0)
(7, 144, 0)
(7, 146, 0)
(7, 148, 0)
(7, 150, 0)
(7, 152, 0)
(7, 154, 0)
(7, 156, 0)
(7, 158, 0)
(7, 160, 0)
(7, 162, 0)
(7, 164, 0)
(7, 166, 0)
(7, 168, 0)
(7, 170, 0)
(7, 172, 0)
(7, 174, 0)
(7, 176, 0)
(7, 178

(8, 4, 0)
(8, 6, 0)
(8, 8, 0)
(8, 10, 0)
(8, 12, 0)
(8, 14, 0)
(8, 16, 0)
(8, 18, 0)
(8, 20, 0)
(8, 22, 0)
(8, 24, 0)
(8, 26, 0)
(8, 28, 0)
(8, 30, 0)
(8, 32, 0)
(8, 34, 0)
(8, 36, 0)
(8, 38, 0)
(8, 40, 0)
(8, 42, 0)
(8, 44, 0)
(8, 46, 0)
(8, 48, 0)
(8, 50, 0)
(8, 52, 0)
(8, 54, 0)
(8, 56, 0)
(8, 58, 0)
(8, 60, 0)
(8, 62, 0)
(8, 64, 0)
(8, 66, 0)
(8, 68, 0)
(8, 70, 0)
(8, 72, 0)
(8, 74, 0)
(8, 76, 0)
(8, 78, 0)
(8, 80, 0)
(8, 82, 0)
(8, 84, 0)
(8, 86, 0)
(8, 88, 0)
(8, 90, 0)
(8, 92, 0)
(8, 94, 0)
(8, 96, 0)
(8, 98, 0)
(8, 100, 0)
(8, 102, 0)
(8, 104, 0)
(8, 106, 0)
(8, 108, 0)
(8, 110, 0)
(8, 112, 0)
(8, 114, 0)
(8, 116, 0)
(8, 118, 0)
(8, 120, 0)
(8, 122, 0)
(8, 124, 0)
(8, 126, 0)
(8, 128, 0)
(8, 130, 0)
(8, 132, 0)
(8, 134, 0)
(8, 136, 0)
(8, 138, 0)
(8, 140, 0)
(8, 142, 0)
(8, 144, 0)
(8, 146, 0)
(8, 148, 0)
(8, 150, 0)
(8, 152, 0)
(8, 154, 0)
(8, 156, 0)
(8, 158, 0)
(8, 160, 0)
(8, 162, 0)
(8, 164, 0)
(8, 166, 0)
(8, 168, 0)
(8, 170, 0)
(8, 172, 0)
(8, 174, 0)
(8, 176, 0)
(8, 178

(9, 4, 0)
(9, 6, 0)
(9, 8, 0)
(9, 10, 0)
(9, 12, 0)
(9, 14, 0)
(9, 16, 0)
(9, 18, 0)
(9, 20, 0)
(9, 22, 0)
(9, 24, 0)
(9, 26, 0)
(9, 28, 0)
(9, 30, 0)
(9, 32, 0)
(9, 34, 0)
(9, 36, 0)
(9, 38, 0)
(9, 40, 0)
(9, 42, 0)
(9, 44, 0)
(9, 46, 0)
(9, 48, 0)
(9, 50, 0)
(9, 52, 0)
(9, 54, 0)
(9, 56, 0)
(9, 58, 0)
(9, 60, 0)
(9, 62, 0)
(9, 64, 0)
(9, 66, 0)
(9, 68, 0)
(9, 70, 0)
(9, 72, 0)
(9, 74, 0)
(9, 76, 0)
(9, 78, 0)
(9, 80, 0)
(9, 82, 0)
(9, 84, 0)
(9, 86, 0)
(9, 88, 0)
(9, 90, 0)
(9, 92, 0)
(9, 94, 0)
(9, 96, 0)
(9, 98, 0)
(9, 100, 0)
(9, 102, 0)
(9, 104, 0)
(9, 106, 0)
(9, 108, 0)
(9, 110, 0)
(9, 112, 0)
(9, 114, 0)
(9, 116, 0)
(9, 118, 0)
(9, 120, 0)
(9, 122, 0)
(9, 124, 0)
(9, 126, 0)
(9, 128, 0)
(9, 130, 0)
(9, 132, 0)
(9, 134, 0)
(9, 136, 0)
(9, 138, 0)
(9, 140, 0)
(9, 142, 0)
(9, 144, 0)
(9, 146, 0)
(9, 148, 0)
(9, 150, 0)
(9, 152, 0)
(9, 154, 0)
(9, 156, 0)
(9, 158, 0)
(9, 160, 0)
(9, 162, 0)
(9, 164, 0)
(9, 166, 0)
(9, 168, 0)
(9, 170, 0)
(9, 172, 0)
(9, 174, 0)
(9, 176, 0)
(9, 178

(10, 4, 0)
(10, 6, 0)
(10, 8, 0)
(10, 10, 0)
(10, 12, 0)
(10, 14, 0)
(10, 16, 0)
(10, 18, 0)
(10, 20, 0)
(10, 22, 0)
(10, 24, 0)
(10, 26, 0)
(10, 28, 0)
(10, 30, 0)
(10, 32, 0)
(10, 34, 0)
(10, 36, 0)
(10, 38, 0)
(10, 40, 0)
(10, 42, 0)
(10, 44, 0)
(10, 46, 0)
(10, 48, 0)
(10, 50, 0)
(10, 52, 0)
(10, 54, 0)
(10, 56, 0)
(10, 58, 0)
(10, 60, 0)
(10, 62, 0)
(10, 64, 0)
(10, 66, 0)
(10, 68, 0)
(10, 70, 0)
(10, 72, 0)
(10, 74, 0)
(10, 76, 0)
(10, 78, 0)
(10, 80, 0)
(10, 82, 0)
(10, 84, 0)
(10, 86, 0)
(10, 88, 0)
(10, 90, 0)
(10, 92, 0)
(10, 94, 0)
(10, 96, 0)
(10, 98, 0)
(10, 100, 0)
(10, 102, 0)
(10, 104, 0)
(10, 106, 0)
(10, 108, 0)
(10, 110, 0)
(10, 112, 0)
(10, 114, 0)
(10, 116, 0)
(10, 118, 0)
(10, 120, 0)
(10, 122, 0)
(10, 124, 0)
(10, 126, 0)
(10, 128, 0)
(10, 130, 0)
(10, 132, 0)
(10, 134, 0)
(10, 136, 0)
(10, 138, 0)
(10, 140, 0)
(10, 142, 0)
(10, 144, 0)
(10, 146, 0)
(10, 148, 0)
(10, 150, 0)
(10, 152, 0)
(10, 154, 0)
(10, 156, 0)
(10, 158, 0)
(10, 160, 0)
(10, 162, 0)
(10, 164, 0